In [142]:
from pulp import value
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [143]:
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from collections import defaultdict
import datetime
from modules.config import (
    PATH_SCENARIOS_REDUCED,
    PATH_DISTANCES,
    PATH_SCENARIO_PROBABILITY,
    PATH_INITIAL_ALLOCATION,
    N_REDUCED_SCNEARIOS,
    ALL_VEHICLE_TYPES,
    PATH_RESULTS_SUMMARY,
)
from modules.stochastic_program.factory import StochasticProgramFactory



### Prepare scenario data
Make scenarios complete cross product

In [39]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [40]:
hex_ids = {*scenarios.index.get_level_values('start_hex_ids').unique()}
hex_ids = list(hex_ids.union({*scenarios.index.get_level_values('end_hex_ids').unique()}))

In [41]:
complete_index = pd.MultiIndex.from_product(
    [
        scenarios.index.get_level_values('scenarios').unique(),
        pd.Index(hex_ids, name="start_hex_ids"),
        pd.Index(hex_ids, name="end_hex_ids"),
        scenarios.index.get_level_values('time').unique(),
        scenarios.index.get_level_values('vehicle_types').unique(),
    ]
)

In [42]:
scenarios = scenarios.reindex(complete_index, fill_value=0)

Find tree structure for non-anticipativity constraints

In [43]:
scenarios_unstacked = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'vehicle_types']) \
             .swaplevel() \
             .sort_index()

In [44]:
node_df = pd.DataFrame(index=scenarios_unstacked.index)
node_df['node'] = 0
node_df['prev_node'] = 0

In [45]:
times = scenarios_unstacked.index.get_level_values('time').unique()
scenario_ids = scenarios_unstacked.index.get_level_values('scenarios').unique()

nodes = []
nodes_counter = 0
groups = [{} for _ in range(len(times))]
for i, time in enumerate(times):
    prev_groups = groups[i-1] if i != 0 else {-1: list(scenario_ids)}

    found_ids = []

    values = scenarios_unstacked.loc[(time)].values

    for scenario_id in scenario_ids:
        if scenario_id in found_ids:
            continue
        current_group = list((values == values[scenario_id]).all(axis=1).nonzero()[0])
        found_ids += current_group
        for prev_group_id, prev_group in prev_groups.items():
            group = [s_id for s_id in current_group if s_id in prev_group]

            if(not group):
                continue

            groups[i][nodes_counter] = group
            for s_id in group:
                node_df.loc[(time, s_id), 'node'] = nodes_counter
                node_df.loc[(time, s_id), 'prev_node'] = prev_group_id
            nodes_counter += 1

## Get Model Parameters


In [46]:
demand = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(dict))))

In [47]:
start = timer()

for _, row in scenarios.reset_index().iterrows():
    demand[row.start_hex_ids][row.end_hex_ids][row.time.hour][row.vehicle_types][row.scenarios] = row.demand

end = timer()
print(f"Successfully converted dataframe into dictionary in {(end - start):.2f} seconds")

Successfully converted dataframe into dictionary in 2.63 seconds


In [48]:
distances = pd.read_pickle(PATH_DISTANCES)

In [49]:
cost = defaultdict(lambda: defaultdict(dict))
profit = defaultdict(lambda: defaultdict(dict))

In [50]:
start = timer()

vehicle_types = list(scenarios.reset_index()['vehicle_types'].unique())

for _, row in distances.reset_index().iterrows():
    for vehicle_type in vehicle_types:
        cost[row.start_hex_id][row.end_hex_id][vehicle_type] = row[f"cost_{vehicle_type}"]
        profit[row.start_hex_id][row.end_hex_id][vehicle_type] = row[f"profit_{vehicle_type}"]

end = timer()

In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
probabilities = pd.read_pickle(PATH_SCENARIO_PROBABILITY)

In [53]:
vehicle_types = ['kick_scooter']
FLEET_CAPACITY = {
    "kick_scooter": 200,
    # "bicycle": 100,
    # "car": 50,
}
factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df, vehicle_types)
factory.set_initial_allocation(FLEET_CAPACITY)

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.02 seconds
_convert_demand finished in 2.56 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 2.59 seconds
_set_max_demand finished in 0.15 seconds
set_initial_allocation finished in 0.00 seconds


In [54]:
stochastic_program = factory.create_stochastic_program()

create_stochastic_program finished in 0.00 seconds


In [55]:
stochastic_program.relocations_disabled = False

In [56]:
stochastic_program.create_model()

create_variables finished in 0.35 seconds
create_objective_function finished in 0.25 seconds
create_demand_constraints finished in 0.16 seconds
create_relocation_binary_constraints finished in 0.01 seconds
create_big_u_sum_constraints finished in 0.03 seconds
create_unfulfilled_demand_binary_constraints finished in 0.01 seconds
create_no_refused_demand_constraints finished in 0.01 seconds
create_max_trips_constraints finished in 0.09 seconds
create_vehicle_movement_constraints finished in 0.11 seconds
create_initial_allocation_constraints finished in 0.00 seconds
create_non_anticipativity_constraints finished in 0.13 seconds
create_constraints finished in 0.55 seconds
create_model finished in 1.33 seconds


In [57]:
stochastic_program.solve(logPath='./log')

/home/moritz/miniconda3/envs/VR/lib/python3.8/site-packages/pulp/apis/coin_api.py:146: UserWarning: `logPath` argument replaces `msg=1`. The output will be redirected to the log file.
  warnings.warn('`logPath` argument replaces `msg=1`. The output will be redirected to the log file.')


Status: Optimal
Optimal Value of Objective Function:  8034.439562028653
Runtime without preprocessing: 0.89 seconds
solve finished in 1.54 seconds


# Benchmarks
## Different Capacities/ Disabled Relocations/ Value Of Perfect Information

In [58]:
capacities = [
    {
        "kick_scooter": 200,
        "bicycle": 100,
        "car": 50,
    },
    # {
    #     "kick_scooter": 1000,
    #     "bicycle": 500,
    #     "car": 250,
    # },{
    #     "kick_scooter": 2000,
    #     "bicycle": 1000,
    #     "car": 500,
    # }
]


In [59]:
results = []

factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df)
factory.include_methods = [None]
for capacity in capacities:
    factory.set_initial_allocation(capacity)

    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ['solve']

    for relocations_disabled in [False, True]:
        for non_anticipativity_disabled in [False, True]:
            stochastic_program.relocations_disabled = relocations_disabled
            stochastic_program.non_anticipativity_disabled = non_anticipativity_disabled
            stochastic_program.create_model()
            stochastic_program.solve()

            results.append({
                **stochastic_program.get_summary(),
                **capacity,
                'relocations_disabled': relocations_disabled,
                'non_anticipativity_disabled': non_anticipativity_disabled,
            })
            print('\n')

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.04 seconds
_convert_demand finished in 2.80 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 2.85 seconds
_set_max_demand finished in 0.16 seconds
Status: Optimal
Optimal Value of Objective Function:  11899.089465475578
Runtime without preprocessing: 2.99 seconds
solve finished in 6.87 seconds


Status: Optimal
Optimal Value of Objective Function:  11941.272872263438
Runtime without preprocessing: 2.32 seconds
solve finished in 6.91 seconds


Status: Optimal
Optimal Value of Objective Function:  2499.226407830746
Runtime without preprocessing: 1.98 seconds
solve finished in 3.16 seconds


Status: Optimal
Optimal Value of Objective Function:  2524.9889157103744
Runtime without preprocessing: 1.66 seconds
solve finished in 2.95 seconds




In [60]:
results_df = pd.DataFrame.from_dict(results)

In [61]:
results_df

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,kick_scooter,bicycle,car,relocations_disabled,non_anticipativity_disabled
0,Optimal,11899.089465,2.987203,12231,7758,1722,2847,200,100,50,False,False
1,Optimal,11941.272872,2.319790,12249,7735,1705,2846,200,100,50,False,True
2,Optimal,2499.226408,1.977206,3954,28519,12846,0,200,100,50,True,False
3,Optimal,2524.988916,1.662327,4008,28401,12792,0,200,100,50,True,True


In [62]:
os.makedirs(os.path.dirname(PATH_RESULTS_SUMMARY), exist_ok=True)
results_df.to_pickle(PATH_RESULTS_SUMMARY)


## Single Modal Benchmark

In [99]:
last_demand = scenarios.copy()

last_demand_unstacked = last_demand.unstack("vehicle_types")
last_demand_unstacked[("demand", "car")] = (
    last_demand_unstacked[("demand", "kick_scooter")]
    + last_demand_unstacked[("demand", "car")]
    + last_demand_unstacked[("demand", "bicycle")]
)
last_demand = last_demand_unstacked.stack('vehicle_types')


In [100]:
FLEET_CAPACITY = {
    "kick_scooter": 200,
    "bicycle": 100,
    "car": 50,
}

results = []

for vehicle_types in [["car"], ["bicycle"], ["kick_scooter"]]:
    current_vehicle_type = vehicle_types[0]
    current_fleet_capacity = {
        current_vehicle_type: FLEET_CAPACITY[current_vehicle_type]
    }

    last_demand.index = last_demand.index.set_levels(
        last_demand.index.get_level_values("vehicle_types").map(lambda x: current_vehicle_type),
        verify_integrity=False,
        level="vehicle_types",
    )

    factory = StochasticProgramFactory(
        last_demand,
        distances,
        probabilities,
        node_df,
        vehicle_types,
        include_methods=[None],
    )
    factory.set_initial_allocation(FLEET_CAPACITY)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    stochastic_program.create_model()
    stochastic_program.solve()
    # we transform the unfulfilled demand of the current lp into the demand for the next lp
    last_demand = stochastic_program.get_unfulfilled_demand().rename(
        columns={"unfulfilled_demand": "demand"}
    )

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": str(vehicle_types)}
    )


Status: Optimal
Optimal Value of Objective Function:  3059.362886891975
Runtime without preprocessing: 0.99 seconds
solve finished in 1.73 seconds
Status: Optimal
Optimal Value of Objective Function:  738.8616900561902
Runtime without preprocessing: 1.02 seconds
solve finished in 1.70 seconds
Status: Optimal
Optimal Value of Objective Function:  2675.4223257152826
Runtime without preprocessing: 1.06 seconds
solve finished in 1.89 seconds


In [101]:
factory = StochasticProgramFactory(
    scenarios,
    distances,
    probabilities,
    node_df,
    ALL_VEHICLE_TYPES,
    include_methods=[None],
)
factory.set_initial_allocation(FLEET_CAPACITY)
stochastic_program = factory.create_stochastic_program()
stochastic_program.include_methods = ["solve"]
stochastic_program.create_model()
stochastic_program.solve()

results.append(
    {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
)


Status: Optimal
Optimal Value of Objective Function:  11899.089465475578
Runtime without preprocessing: 3.33 seconds
solve finished in 7.49 seconds


In [102]:
results = pd.DataFrame.from_dict(results)
results

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,vehicle_types
0,Optimal,3059.362887,0.991741,2051,8440,36,313,['car']
1,Optimal,738.861690,1.023856,4034,4406,92,674,['bicycle']
2,Optimal,2675.422326,1.059601,3678,728,0,5922,['kick_scooter']
3,Optimal,11899.089465,3.330187,12231,7758,1722,2847,"[kick_scooter, bicycle, car]"


In [103]:
compare = results.iloc[[3]]
compare.append(results.iloc[range(3)].sum(), ignore_index=True)

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,vehicle_types
0,Optimal,11899.089465,3.330187,12231,7758,1722,2847,"[kick_scooter, bicycle, car]"
1,OptimalOptimalOptimal,6473.646903,3.075198,9763,13574,128,6909,['car']['bicycle']['kick_scooter']


## Value Of The Stochastic Solution

In [148]:
demand = scenarios.copy()
demand = (
    demand.unstack("scenarios")["demand"]
    .sum(axis=1)
    .to_frame()
    .rename(columns={0: "demand"})
)
demand["scenarios"] = 0
demand = demand.set_index("scenarios", append=True).reorder_levels(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)
demand['demand'] = demand['demand'] / N_REDUCED_SCNEARIOS
demand['floored'] = demand['demand'].apply(np.floor)
demand['ceiled'] = demand['demand'].apply(np.ceil)


In [ ]:
results = []
FLEET_CAPACITY = {
    "kick_scooter": 200,
    "bicycle": 100,
    "car": 50,
}

In [ ]:
for rounding_mode in ["floored", "ceiled"]:
    factory = StochasticProgramFactory(
        demand[[rounding_mode]].rename(columns={rounding_mode: "demand"}),
        distances,
        probabilities,
        node_df,
        ALL_VEHICLE_TYPES,
        include_methods=[None],
    )
    factory.set_initial_allocation(FLEET_CAPACITY)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]

    # assign all weight to first scenario
    stochastic_program.weighting = {0: 1}

    # discard non-anticipativity constraints
    stochastic_program.non_anticipativity_disabled = True

    stochastic_program.create_model()
    stochastic_program.solve()

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
    )
